In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import random
import glob
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing

In [ ]:
def load_data(filename):
    f = open(filename, 'r')
    csv_r = csv.reader(f, delimiter='\t')
    data = []
    for row in csv_r:
        data.append([float(elem) for elem in row])
    return  np.array(data)

In [ ]:
# Load Data: data is already normalized
X = load_data('X_train.txt')
print(X.shape)
print(len(X))

# load labels
y = load_data(('y_train.txt'))
y = np.ravel(y)
print(y.shape)

In [ ]:
# Select a balanced portion of vessel/background pixels dataset to tune parameters
all_idx = list(range(len(X)))
idx_ves = list(np.ravel(np.where(y == 1)))
#print(idx_ves.shape)

idx_back = list(set(all_idx)-set(idx_ves))
print(len(idx_back))
print(len(idx_ves))

In [ ]:
ves_tuning = random.sample(idx_ves, 101404) 
back_tuning = random.sample(idx_back, 101404)
idx_training = ves_tuning + back_tuning

In [ ]:
MD_neigh = KNeighborsClassifier(n_neighbors=5)

In [ ]:
MD_neigh.fit(X[idx_training,:], y[idx_training])
print(MD_neigh)

In [ ]:
# save the model to disk
filename = 'knn_model.sav'
pickle.dump(MD_neigh, open(filename, 'wb'))

In [ ]:
#Load test 
# Load Data
X = load_data('X_test.txt')
print(X.shape)
print(len(X))

y = load_data(('y_test.txt'))
y = np.ravel(y)
print(y.shape)

In [ ]:
#load the model from disk
loaded_model = pickle.load(open( 'knn_model.sav', 'rb'))
result = loaded_model.score(X, y)
print(result)

In [ ]:
predict = loaded_model.predict(X)
print(predict)

In [ ]:
# Create output folder
output = 'output_knn'
os.mkdir(output)

In [ ]:
list_name = list()
images_path = '../../filter/filter_test'
for infile in glob.glob(images_path + '/*.png'):
    p, im_name = os.path.split(infile)
    file, ext = os.path.splitext(im_name)
    list_name.append(file)
    
    
j = 0    
for num, i in enumerate(range(0,len(predict), 8281)):
    im =  np.asarray(predict[j:i+8281])
    im = im.reshape((91,91))
    cv2.imwrite(output + '/'+list_name[num] + '.png' , im*255)
    j = i+8281
    